# Inverse Kinematics

In this tutorial, we demonstrate how to use MAP-Elites to solve an inverse kinematics problem.

## Overview

Inverse kinematics is an important robotics problem in which, given a final robot configuration, one must figure out the actions to take to accomplish that position. (TODO: update this)

In this example, we will deal with a planar (2D) 12-DoF arm that needs to move its end effector to a certain position. Using MAP-Elites, we will find joint configurations that move the arm to a wide variety of positions.

## Setup

In [1]:
import sys
!{sys.executable} -m pip install ribs[examples]

In [6]:
import matplotlib.pyplot as plt
import numpy as np

from ribs.archives import GridArchive
from ribs.optimizers import Optimizer

## Visualization

In [3]:
def visualize(angles, joint_lengths):
    return

## Behavior Characteristics

In [4]:
def forward_kinematics(angles, joint_lengths):
    return

## Evaluation Function

Perhaps how straight the robot arm is? See the IsoLineDD paper.

## Running MAP-Elites

In [7]:
opt = Optimizer()

TypeError: __init__() missing 3 required positional arguments: 'x0', 'sigma0', and 'archive'

## Analyzing Results

Here, we will print out all the solutions we found.

## Conclusion

In this tutorial, we...